<center> <h1> 04. Betas <h1> <center>

This notebook looks at some way to estimate the variance covariance matrix of a portfolio. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

x    = pd.read_csv("FFmFactorsPs.csv")
Rme  = x.iloc[:,1]                #market excess return
RSMB = x.iloc[:,2]                #small minus big firms
RHML = x.iloc[:,3]                #high minus low book-to-market ratio
Rf   = x.iloc[:,4]                #interest rate

x  = pd.read_csv("FF25Ps.csv", header = None) 
# Returns for 25 FF portfolios
R  = x.iloc[:,1:]                     
for col in R.columns:
    R[col] = R[col] - Rf
# Use just 5 assets to make the printing easier
Re = R.iloc[:,[0,6,12,18,24]]          

## 4.1. Covariance Matrix with Average Correlations

For the diagonal element of the estimated matrix $\Sigma$ use the historical variances. Correct the covariances by assuming the structure:

$$ \sigma_{ij} = \bar{\rho}\sqrt{\sigma^2_j \sigma^2_i} $$

where $\bar{\rho}$ is the average correlation of the portfolio stocks. One can estimate the matrix by:

$$ M = np.ones(n,n)*\bar{\rho} + (1-\bar{\rho})*I_n \\
 \Sigma  = (\sigma*\sigma^T) \circ M$$

For $\sigma$ the vector of standard deviations and $\circ$ the element wise product (Hadamard product).

In [ ]:
# Standard deviations
sigma = Re.std(ddof=1).to_numpy()           

# correlation matrix
Corr_matrix = np.corrcoef(Re, rowvar=False) 

# compute average of unique off-diagonal correlations (upper triangle, k=1)
n = Corr_matrix.shape[0]
off_diag_idx = np.triu_indices(n, k=1)
phi_hat = Corr_matrix[off_diag_idx].mean()

# build matrix of sigma products
M = np.outer(sigma, sigma)

# constant-correlation covariance matrix
CC = phi_hat * M
np.fill_diagonal(CC, sigma**2)

# print
print("Cov matrix (constant corr):\n", CC)

Cov matrix (constant corr):
 [[73.4754298  39.48310678 33.43335941 32.42241325 32.50918271]
 [39.48310678 37.3705188  23.84369254 23.12271535 23.18459679]
 [33.43335941 23.84369254 26.79578746 19.57976755 19.63216728]
 [32.42241325 23.12271535 19.57976755 25.19980408 19.03853672]
 [32.50918271 23.18459679 19.63216728 19.03853672 25.33486492]]


## 4.2. Covariance Matrix from a Single-Index Model


The single-index model is another way to reduce the number of parameters that we need to estimate in order to construct the covariance matrix of assets. The model assumes that the co-movement between assets is due to a single common influence (here denoted $R_{mt}$).
If (4.2) and (4.8) are true, then the variance of asset $i$ and the covariance of assets $i$
and $j$ are
\begin{align}
\sigma_i^2 &= \beta_i^2 \, \sigma_{m}^2 + \operatorname{Var}(\varepsilon_{it}),  \\
\sigma_{ij} &= \beta_i \beta_j \, \sigma_{m}^2 \quad \text{for } i \neq j
\end{align}
where $\sigma_{m}^2$ is the variance of $R_m$ and $\beta$ is the regression slope. 


In [ ]:
##### STEP 1: compute the betas
import statsmodels.api as sm

# Obs and number of periods
T, n = Re.shape

# Add constant + market excess return as regressors
X = sm.add_constant(Rme) 

betas = []
var_res = []

# run regression asset by asset
for col in Re.columns:
    y = Re[col].to_numpy()
    model = sm.OLS(y, X).fit()
    betas.append(model.params[1])       
    var_res.append(model.resid.var(ddof=1))

betas = np.array(betas)
var_res = np.array(var_res)

# Put into DataFrame for nice display
colNames = [f"asset {i+1}" for i in range(n)]
results = pd.DataFrame([betas], index=["β on Rme"], columns=colNames)

print(f"β for {n} assets, from OLS of Re on constant and Rme:")
print(results)

β for 5 assets, from OLS of Re on constant and Rme:
           asset 1   asset 2   asset 3   asset 4   asset 5
β on Rme  1.341049  1.169075  0.994488  0.942687  0.849429


C:\Users\jraus\AppData\Local\Temp\ipykernel_3436\3075256537.py:19: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  betas.append(model.params[1])       # coefficient on Rme


In [43]:
##### STEP 2: Construct the covariance matrix
var_rme = Rme.var(ddof=1)


def IndexCovMatrix(betas,var_rme,var_res):
    n = len(betas)
    Cov_matrix = np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            if i == j:
                Cov_matrix[i,j] = betas[i]**2 * var_rme + var_res[i]
            else:
                Cov_matrix[i,j] = betas[i]*betas[j]*var_rme 
    return Cov_matrix

Cov_matrix = IndexCovMatrix(betas,var_rme,var_res)
print("Cov matrix (Signle Index model):\n", Cov_matrix)

Cov matrix (Signle Index model):
 [[73.4754298  33.23221395 28.26939986 26.79689574 24.14592034]
 [33.23221395 37.3705188  24.6441849  23.36051196 21.04949269]
 [28.26939986 24.6441849  26.79578746 19.87191267 17.90601513]
 [26.79689574 23.36051196 19.87191267 25.19980408 16.97332179]
 [24.14592034 21.04949269 17.90601513 16.97332179 25.33486492]]


## 4.3. Covariance Matrix from a Multi-Index Model

A multi-index model is based on

$$ R_{it} = a_i + b'_i I_t + \varepsilon_{it}$$

The entries of the variance matirx $\Sigma$ are given by: 
$$ \sigma_{ij} = b'_i \Omega b_j + Cov(\varepsilon_{it}, \varepsilon_{jt}) $$

for $\Omega$ the variance matrix of the index vector. $Cov(\varepsilon_{it}, \varepsilon_{jt}) = 0$ if $j \ne i$ 

In [84]:
# Regressors
I = pd.concat([Rme, RSMB, RHML], axis=1)
X = sm.add_constant(I)   
T, n = Re.shape
k = X.shape[1]

betas = np.zeros((k-1, n))    
var_res = np.zeros(n)

for j, col in enumerate(Re.columns):
    y = Re[col].to_numpy()
    model = sm.OLS(y, X).fit()
    betas[:, j] = model.params[1:]          
    var_res[j] = model.resid.var(ddof=1) 

# Put results in a DataFrame
colNames = [f"asset {j+1}" for j in range(n)]
rowNames = ["β on Rme", "β on RSMB", "β on RHML"]
β_df = pd.DataFrame(betas, index=rowNames, columns=colNames)

print("OLS slope coefficients:")
print(β_df)


OLS slope coefficients:
            asset 1   asset 2   asset 3   asset 4   asset 5
β on Rme   1.069980  1.080000  1.034791  1.056347  1.040888
β on RSMB  1.263934  0.768144  0.436871  0.152616 -0.087656
β on RHML -0.277733  0.160222  0.486654  0.603010  0.770033


In [105]:
T,n = Re.shape
Cov_matrix = np.zeros((n,n))
Omega = I.cov()

# Rows
for i in range(n):
    for j in range(n):
        if i == j:
            Cov_matrix[i,j] =  betas[:,i] @ Omega @ betas[:,j] + var_res[j]
        else:
            Cov_matrix[i,j] =  betas[:,i] @ Omega @ betas[:,j]


print("Cov matrix (Multi Index model):\n", Cov_matrix)

Cov matrix (Multi Index model):
 [[73.4754298  41.84732875 31.05027761 25.44878901 18.93983584]
 [41.84732875 37.3705188  27.38401837 24.14082747 20.1454443 ]
 [31.05027761 27.38401837 26.79578746 22.23872734 20.10865365]
 [25.44878901 24.14082747 22.23872734 25.19980408 20.71697708]
 [18.93983584 20.1454443  20.10865365 20.71697708 25.33486492]]


## 4.4. Shirnkage

The historical sample covariance matrix, $S$, can exhibit significant noise in small samples. One way of handling that is to “shrink” the sample covariance matrix towards a target, $F$. 
$$ \Sigma = \theta F + (1 - \theta)S \text{ , for } 0 \le \theta \le 1 $$

One can use one of the matrix computed above as target.

In [ ]:
# Compute the historical cov matrix
S = Re.cov()

# Shrink toward avergae corr matrix
theta = 0.6
Sigma = CC * theta + (1-theta) * S

print("Cov matrix (Shrinked):\n", Sigma)

Cov matrix (Shrinked):
            1          7          13         19         25
1   73.475430  40.899273  31.848757  29.458456  27.580683
7   40.899273  37.370519  25.935208  23.928022  22.473384
13  31.848757  25.935208  26.795787  21.200679  20.171827
19  29.458456  23.928022  21.200679  25.199804  20.036265
25  27.580683  22.473384  20.171827  20.036265  25.334865
